In [ ]:
import tensorflow as tf # v2.9.1
import pandas as pd # v1.5.3
import numpy as np # v1.22.3
import matplotlib.pyplot as plt
import os
import datetime
import random
import time
import sklearn # v1.0.1

In [ ]:
print(tf.__version__)
print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)

In [ ]:
# get all csv files for a given patient
def find_filenames(path, name_filter='patientA', suffix=".csv"):
    fileset = []
    
    for root, dirs, files in os.walk(path):
        for file in files:
                if file.endswith(suffix) and name_filter in file[:len(name_filter)] and file[len(name_filter)] == '_':
                        fileresult = os.path.join(root, file)
                        fileset.append(fileresult)
    
    return fileset


def split_sequences_new(sequences, n_steps_in=12, n_steps_out=12):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        x_end_ix = i + n_steps_in
        y_end_ix = x_end_ix + n_steps_out
        # check if we are beyond the dataset
        if y_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern (include control actions in y's)
        seq_x, seq_y = sequences[i:x_end_ix, :], sequences[x_end_ix:y_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# determines the bounds used by the semantic loss function
def find_bounds(BG, IOB=None, interval=0.95):
    lower_bounds = [] # bg, iob, bg', iob', bg'', iob''
    upper_bounds = []

    l, u = confidence_interval(BG, interval=interval)
    lower_bounds.append(l)
    upper_bounds.append(u)

    if IOB is not None:
        l, u = confidence_interval(IOB, interval=interval)
        lower_bounds.append(l)
        upper_bounds.append(u)

    l, u = confidence_interval(np.diff(BG), interval=interval)
    lower_bounds.append(l)
    upper_bounds.append(u)

    if IOB is not None:
        l, u = confidence_interval(np.diff(IOB), interval=interval)
        lower_bounds.append(l)
        upper_bounds.append(u)

    l, u = confidence_interval(np.diff(BG, n=2), interval=interval)
    lower_bounds.append(l)
    upper_bounds.append(u)

    if IOB is not None:
        l, u = confidence_interval(np.diff(IOB, n=2), interval=interval)
        lower_bounds.append(l)
        upper_bounds.append(u)

    return lower_bounds, upper_bounds

def confidence_interval(x, interval=0.95):
    # try:
    #     gamma_a, gamma_loc, gamma_scale = stats.gamma.fit(x)
    #     ci = stats.gamma.interval(interval, gamma_a, gamma_loc, gamma_scale)
    # except:
    mean = np.mean(x)
    std = np.std(x)
    l = mean - 1.96*std
    u = mean + 1.96*std
    ci = (l, u)
    return ci

def find_outliers(dbg, k):
    q1 = np.quantile(dbg, 0.25)
    q3 = np.quantile(dbg, 0.75)
    IQR = q3 - q1
    outliers = (dbg < (q1 - k*IQR)) | (dbg > (q3 + k*IQR))
    return np.where(outliers)


In [ ]:
# read in files from filelist
# FYI: nosplit refers to the fact that this function doesn't split the data into
# sequences of a fixed size; that is done in split_sequences after the data is already read.
# I did this so to split the files into train, CV, and test sets more easily.
def read_files_nosplit(filelist, features, min_length=24, k=1.5, verbose=True):
    feature_cols = features

    X = np.array([])
    traces = np.array([])
    t = 0

    for i, file in enumerate(filelist):
        df = pd.read_csv(file)
        df['Time'] = pd.to_datetime(df['Time'])

        if len(df) == 0: # as a byproduct of the way I processed files, at most 1 csv is empty per patient
            if verbose:
                print(f"Trace {i} is empty")
            continue
        
        # normally only drop the first 3 hours, but if a carbohydrate is consumed, drop next 3 hours as well
        start = df.loc[0, 'Time']
        end = df.loc[0, 'Time'] + datetime.timedelta(hours=3)
        to_drop = df[np.all([df["Time"] >= start, df["Time"] < end], axis=0)].index
        also_drop = df[np.all([df['Time'].dt.hour > 7, df['Time'].dt.hour < 19], axis=0)].index
        to_drop = set(to_drop).union(also_drop)

        for i in range(1, len(df)):
            if df.loc[i, 'Carbs'] > 0:
                start = df.loc[i, 'Time']
                end = df.loc[i, 'Time'] + datetime.timedelta(hours=3)
                temp = df[np.all([df["Time"] >= start, df["Time"] < end], axis=0)].index
                to_drop = to_drop.union(temp)

        df.drop(to_drop, inplace=True)

        if len(df) < min_length:
            if verbose:
                print(f'Not enough data in trace {i} away from carbs')
            continue

        # if (df['CGM_glucose'] < 70).sum() > 0:
        #     if verbose:
        #         print(f"Patient entered hypoglycemia in trace {i}, treated with rescue carbs")
        #     continue

        # find outliers by the difference in BG from i to i+1
        slopes = np.diff(df['CGM_glucose'])
        outliers_idx = df.iloc[find_outliers(slopes, k)].index + 1

        # split df along continuous segments
        segments = []
        indices = df.index.to_list()
        seg = [indices[0]]

        i = 1
        while i < len(indices):
            if indices[i] == (seg[-1]+1):
                if indices[i] in outliers_idx:
                    segments.append(seg)
                    seg = []
                    i += 1
                    if i < len(indices):
                        seg.append(indices[i])
                        i += 1
                else:
                    seg.append(indices[i])
                    i += 1
            else:
                segments.append(seg)
                seg = [indices[i]]
                i += 1
        segments.append(seg)

        if verbose:
            print(f"{len(segments)} segment(s) in trace {i} with {len(outliers_idx)} outliers")

        for seg in segments:
            if len(seg) < min_length:
                if verbose:
                    print(f"Not enough data in this segment of trace {i}")
                continue

            seg_df = df.loc[seg, :]

            time_increments = np.unique(np.diff(seg_df['Time'])/np.timedelta64(1, 's'))

            if len(time_increments) > 1 or time_increments[0] != 300:
                if verbose:
                    print(f'Timesteps not always 5 min apart for this segment of trace {i}')
                continue

            # time of day in minutes from 12:00 AM, scaled to be [0, 1)
            seg_df['time_of_day'] = (60*seg_df['Time'].dt.hour + seg_df['Time'].dt.minute)/1440

            trace_num = t*np.ones(shape=(len(seg_df)))
            t += 1
            seq = np.c_[seg_df.loc[:, feature_cols]]

            if seq.shape[0] > 0:
                if len(X)>0:
                    X=np.r_[X,seq]
                    traces=np.r_[traces, trace_num]
                else:
                    X=seq
                    traces = trace_num

    return X, traces 
    
# splits the processed data into input and output sequences of fixed lengths
# the input data will be fed into the model, and the output data will be used to 
# judge how accurate the model is
def split_sequences(X, traces, steps_in=12, steps_out=12):
    X_ = np.array([])
    y_ = np.array([])
    for i in range(int(traces[-1])):
        seq = X[np.where(traces == i)]

        if len(seq) > 0:
            Xtemp, ytemp = split_sequences_new(seq, n_steps_in=steps_in, n_steps_out=steps_out)

            if Xtemp.shape[0] > 0:
                if len(X_)>0:
                    X_=np.r_[X_,Xtemp]
                    y_=np.r_[y_,ytemp]
                else:
                    X_=Xtemp
                    y_=ytemp

    return X_, y_

# X, y, traces = read_files_nosplit(filelist)

# X, y = split_sequences(X, y, traces)
# print(X.shape, y.shape)

In [ ]:
# process PSO3 data
def read_data_pso3(filelist, scaler, features, steps_in=12, steps_out=12, train_set=True, n_states=1, k=1.5, random_state=None):
    X, traces = read_files_nosplit(filelist, features, min_length=(steps_in+steps_out), k=k, verbose=False)
    if train_set:
        print('Processed non-sequenced train data:', X.shape)
        X = scaler.fit_transform(X)
    else:
        print('Processed non-sequenced test data:', X.shape)
        X = scaler.transform(X)

    if 'IOB' in features and n_states > 1:
        L, U = find_bounds(X[:, 0], X[:, 1])
    else:
        L, U = find_bounds(X[:, 0])

    X, y = split_sequences(X, traces, steps_in=steps_in, steps_out=steps_out)
    return X, y, L, U


In [ ]:
def shape_bounds(L, U, steps_out=12):
    L = tf.convert_to_tensor([L], dtype='float32')
    U = tf.convert_to_tensor([U], dtype='float32')
    L = tf.repeat(L, axis=0, repeats=steps_out)
    U = tf.repeat(U, axis=0, repeats=steps_out)
    return L, U

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, LSTM, Input, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hidden_dim, n_features, n_states, drop_rate=0.2):
    enc_inp = Input(shape=(None, n_features), name='enc_input')
    dec_inp = Input(shape=(None, n_features), name='dec_input')

    if drop_rate > 0:
        drop1 = Dropout(drop_rate)
        drop2 = Dropout(drop_rate)

    enc = LSTM(hidden_dim, return_state=True, name='encoder')
    enc_out, state_h, state_c = enc(enc_inp)
    if drop_rate > 0:
        state_h, state_c = drop1([state_h, state_c])
    enc_state = [state_h, state_c]

    dec = LSTM(hidden_dim, return_sequences=True, return_state=True, name='decoder')
    dec_out, _, _ = dec(dec_inp, initial_state=enc_state)
    if drop_rate > 0:
        dec_out = drop2(dec_out)

    ffn = TimeDistributed(Dense(n_states, activation='linear'), name='dense')
    out = ffn(dec_out)

    model = Model(inputs=[enc_inp, dec_inp], outputs=out)

    encoder_model = Model(enc_inp, enc_state)

    dec_inp_h = Input(shape=(hidden_dim,))
    dec_inp_c = Input(shape=(hidden_dim,))
    dec_state_inp = [dec_inp_h, dec_inp_c]
    dec_out, dec_h, dec_c = dec(dec_inp, initial_state=dec_state_inp)
    dec_states = [dec_h, dec_c]
    dec_out = ffn(dec_out)

    all_dec_inp = [dec_inp] + dec_state_inp # concat operation for python lists: [1] + [2] = [1, 2]
    all_dec_out = [dec_out] + dec_states
    decoder_model = Model(all_dec_inp, all_dec_out)

    return model, encoder_model, decoder_model

In [ ]:
def train_model(model, Xtrain, ytrain, optimizer, loss_func, epochs, batches, n_states):
    train_loss = tf.keras.metrics.Mean()
    train_mse = tf.keras.metrics.Mean()
    train_batches = tf.data.Dataset.from_tensor_slices((Xtrain, ytrain)).batch(batches)

    
    @tf.function
    def train_step(x, dec_inp, y):
        # forward
        with tf.GradientTape() as tape:
            yhat = model([x, dec_inp], training=True)
            # loss = semantic_loss_new(y, yhat)
            loss = loss_func(y, yhat)
        
        # backward
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # metrics
        train_loss(loss)
        train_mse(loss_func(y, yhat))
        return loss

    for epoch in range(epochs):
        if epoch == 100:
            lr = 3e-5
            optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        if epoch == 200:
            lr = 1e-5
            optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        start = time.time()
        train_loss.reset_states()
        train_mse.reset_states()

        for batch, (x, y) in enumerate(train_batches):    
            # create decoder input
            dec_init = tf.expand_dims(x[:, -1], 1) # (batch, 1, 3)
            dec_rest = y[:, :-1, :]
            dec_inp = tf.concat([dec_init, dec_rest], axis=1)
            
            y_states = y[:, :, :n_states]
            
            loss = train_step(x, dec_inp, y_states)

            # if batch % 250 == 0:
            #     print(f'Batch {batch} Loss {train_loss.result():.4f} ') # MSE {train_mse.result():.4f}
        
        if epoch == 0 or  (epoch+1)% 10 == 0:
            print(f'Epoch {epoch+1} Train Loss {train_loss.result():.4f} Train MSE {train_mse.result():.4f} in {time.time() - start:.3f}') 

In [ ]:
def eval_prediction(x, y, encoder_model, decoder_model, features, steps_out=24, n_states=1):
    n_inputs = len(features) - n_states
    states_value = encoder_model(x, training=False)

    outputs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    dec_init = x[:, -1:, :]

    for i in range(steps_out):
        yhat_t, h, c = decoder_model([dec_init] + states_value, training=False)
        # yhat2 = model([x, dec_init], training=False)
        
        dec_init = tf.concat([yhat_t, tf.cast(y[:, i:i+1, -n_inputs:], dtype='float32')], axis=2)
        outputs = outputs.write(i, yhat_t)
        # print(dec_init, y[:, i, :])
        states_value = [h, c]
        
    yhat = tf.transpose(outputs.stack(), perm=[1, 0, 3, 2])
    yhat = tf.reshape(yhat, shape=(-1, steps_out, n_states))
    return yhat

def untransform(y, yhat, scaler, features, steps_out=12, n_states=1):
    n_inputs = len(features) - n_states
    y = tf.cast(y, dtype='float32')
    ypred = np.array([])
    ytrue = np.array([])

    for i in range(steps_out):
        bg_pred = yhat[:, i, :1]
        bg = y[:, i, :1]

        fake_inputs = tf.zeros(shape=(y.shape[0], n_inputs)) # don't need time_of_day since it isn't included in the scaler
        if 'IOB' in features and n_states > 1:
            iob_pred = yhat[:, i, 1:2]
            iob = y[:, i, 1:2]

            pred = tf.concat([bg_pred, iob_pred, fake_inputs], axis=1)
            true = tf.concat([bg, iob, fake_inputs], axis=1)
        else:
            pred = tf.concat([bg_pred, fake_inputs], axis=1)
            true = tf.concat([bg, fake_inputs], axis=1)
        
        state_pred = tf.expand_dims(scaler.named_transformers_['scaler'].inverse_transform(pred)[:, :n_states], 1)
        state = tf.expand_dims(scaler.named_transformers_['scaler'].inverse_transform(true)[:, :n_states], 1)

        if len(ypred)>0:
            ypred = tf.concat([ypred, state_pred], axis=1)
            ytrue = tf.concat([ytrue, state], axis=1)
        else:
            ypred = state_pred
            ytrue = state
            
    return ytrue, ypred

def rmse_across_time(y, yhat):
    root_mean_squared_error = tf.keras.metrics.RootMeanSquaredError()
    ret = []

    for t in range(y.shape[1]):
        root_mean_squared_error.reset_states()
        bg_t = y[:, t, 0]
        bghat_t = yhat[:, t, 0]

        rmse_t = root_mean_squared_error(bg_t, bghat_t)
        ret.append(rmse_t.numpy())
    
    return ret

def performance_results(y, yhat):
    np.set_printoptions(suppress=True)

    mean = np.mean(y - yhat, axis=0)
    std = np.std(y - yhat, axis=0)

    print('Mean and St. Dev. of Model Error over time:')
    print(np.c_[mean, std])

    rmse = tf.keras.metrics.RootMeanSquaredError()
    ret_rmse = rmse(y, yhat).numpy()
    print('Overall RMSE:', ret_rmse)

    print('RMSE across time:')
    print(rmse_across_time(y, yhat))
    return ret_rmse


In [ ]:
def evaluate_model(encoder_model, decoder_model, Xtest, ytest, scaler, features, steps_out=12, n_states=1):
    test_ds = tf.data.Dataset.from_tensor_slices((Xtest, ytest)).batch(1028)
    test_rmse = tf.keras.metrics.Mean()
    test_rmse.reset_states()
    root_mean_squared_error = tf.keras.metrics.RootMeanSquaredError()

    y_true = np.array([]) # ending shape is (n_samples, n_steps, n_states)
    y_pred = np.array([])

    for x, y in test_ds:
        # yhat = attn_eval_prediction(x, y, steps_out=steps_out)
        yhat = eval_prediction(x, y, encoder_model, decoder_model, features, steps_out=steps_out, n_states=n_states)
        print(yhat.shape)
        
        y_states, yhat = untransform(y, yhat, scaler, features, steps_out=steps_out, n_states=n_states)
        print(y_states.shape, yhat.shape)


        rmse = root_mean_squared_error(y_states, yhat)
        test_rmse(rmse)
        # print('.', end='')

        if len(y_true) > 0:
            # print(y_true.shape, y_states.shape)
            y_true = np.r_[y_true, y_states]
            y_pred = np.r_[y_pred, yhat]
        else:
            y_true = y_states
            y_pred = yhat
            # print(y_true)

    print(y_true.shape, y_pred.shape)
    res = test_rmse.result().numpy()
    print("Test RMSE:", res)
    print('BG Results')
    res2 = performance_results(y_true[:, :, :1], y_pred[:, :, :1])
    print('IOB Results')
    res3 = performance_results(y_true[:, :, -1:], y_pred[:, :, -1:])
    return res, res2, res3

In [ ]:
def graph_traces(enc, dec, X, y, features, scaler, patient, steps_in=24, steps_out=24, n_states=1):
    for _ in range(10):
        i = np.random.randint(low=0, high=len(X)-1)
        x_ = tf.expand_dims(X[i], 0)
        y_ = tf.expand_dims(y[i], 0)

        yhat = eval_prediction(x_, y_, enc, dec, features, steps_out, n_states)
        y_, yhat = untransform(y_, yhat, scaler, features, steps_out)
        x_ = x_[:, :, :2]
        x_, _ = untransform(x_, np.zeros_like(x_), scaler, features, steps_in)

        fig, ax = plt.subplots()
        ax2 = ax.twinx()

        ax.plot(range(steps_in), x_[0, :, 0], 'o--', label='BG input')
        ax.plot(range(steps_in, steps_in+steps_out), y_[0, :, 0], 'o--', label='BG true')
        ax.plot(range(steps_in, steps_in+steps_out), yhat[0, :, 0], 'o--', label='BG pred')

        if 'IOB' in features:
            ax2.plot(range(steps_in), x_[0, :, 1], 'ro--', label='IOB input')
            ax2.plot(range(steps_in, steps_in+steps_out), y_[0, :, 1], 'ro--', label='IOB true')
            ax2.plot(range(steps_in, steps_in+steps_out), yhat[0, :, 1], 'yo--', label='IOB pred')
            ax2.legend(loc='lower left')

        ax.legend(loc='upper left')

        path = './results/plots/'+patient
        if not os.path.exists(path):
            os.makedirs(path)
        plt.savefig(path+'/trace_'+str(i))
        plt.clf()
        
        

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer

path = "/Users/liuyushen/Desktop/BG Prediction/data/openAPS_patient"
# patient = 'patient'+str(2)
features = ['CGM_glucose', 'rate']
steps_in = 12
steps_out = 6
random_state=41
scaled = True
hidden_dim=64
drop_rate=0

### training parameters ###
EPOCHS = 250
BATCHES = 64

# scaled: lr = 5e-4, epochs = 250
# unscaled: lr = 1e-3, epochs = 200+
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
base_loss = tf.keras.losses.MeanSquaredError()
omega=50
n_states=1




In [ ]:
# input is ( true, predicted )
def semantic_loss_new(x, xhat):
    mse = base_loss(x, xhat)

    dxhat = tf.concat((tf.zeros(shape=(xhat.shape[0], 1, n_states)), tf.experimental.numpy.diff(xhat, axis=1)), axis=1) # (batch, steps_in, n_features)
    d2xhat = tf.concat((tf.zeros(shape=(xhat.shape[0], 2, n_states)), tf.experimental.numpy.diff(xhat, axis=1, n=2)), axis=1)
    Shat = tf.concat([xhat, dxhat, d2xhat], axis=2)

    reg = tf.reduce_any(tf.logical_or(tf.less(Shat, L), tf.greater(Shat, U)), axis=2) # rule violated in timestep i?
    reg = tf.cast(reg, dtype='float32')

    S = tf.minimum(tf.maximum(Shat, L), U)
    
    dist = tf.norm(S - Shat, ord='euclidean', axis=-1)
    sem = omega*tf.reduce_mean(reg*dist)

    return mse + sem

In [ ]:
for patient_idx in range(26, 51):
    patient = 'patient'+str(patient_idx)
    print(f"##### Starting {patient} #####")
    all_rmse = []
    bg_rmse = []
    iob_rmse = []

    filelist = np.array(find_filenames(path, patient))
    if len(filelist) == 0:
        print(f'No data for {patient} \n')
        continue

    kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    for cv, (train_idx, test_idx) in enumerate(kfold.split(filelist)):
        print(f"--- CV {cv+1} ---")
        filelist_train = filelist[train_idx]
        filelist_test = filelist[test_idx]

        if 'time_of_day' in features:
            scaler = ColumnTransformer([('scaler', StandardScaler(), [i for i in range(len(features)-1)])], remainder='passthrough') # doesn't scale time of day
        else:
            scaler = ColumnTransformer([('scaler', StandardScaler(), [i for i in range(len(features))])], remainder='passthrough')
        Xtrain, ytrain, L, U = read_data_pso3(filelist_train, scaler, features, steps_in, steps_out, train_set=True, n_states=n_states, random_state=random_state)
        L, U = shape_bounds(L, U, steps_out)
        Xtest, ytest, _, _ = read_data_pso3(filelist_test, scaler, features, steps_in, steps_out, train_set=False, random_state=random_state)
        print(Xtrain.shape, Xtest.shape)
        print(ytrain.shape, ytest.shape)

        if len(Xtrain) <= 0 or len(Xtest) <= 0:
            continue

        model, encoder_model, decoder_model = build_model(hidden_dim, len(features), n_states, drop_rate=drop_rate)

        print('Begin training...')
        train_model(model, Xtrain, ytrain, optimizer, base_loss, EPOCHS, BATCHES, n_states)
        print('Begin evaluation...')
        cv_rmse_, bg_rmse_, iob_rmse_ = evaluate_model(encoder_model, decoder_model, Xtest, ytest, scaler, features, steps_out, n_states)
        all_rmse.append(cv_rmse_)
        bg_rmse.append(bg_rmse_)
        iob_rmse.append(iob_rmse_)
        
        print('\n')
    
    # graph_traces(encoder_model, decoder_model, Xtest, ytest, features, scaler, patient, steps_in, steps_out, n_states)

    print(f'{patient} average RMSE across {cv+1} CV sets:', np.mean(all_rmse), '(BG:', np.mean(bg_rmse), ', IOB:', np.mean(iob_rmse), ')')
    print()